<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/107_%E5%91%BC%E3%81%B3%E5%87%BA%E3%81%97%E5%8F%AF%E8%83%BD%E5%9E%8B%E3%81%A8%E3%83%87%E3%82%B3%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

呼び出し可能型とデコレーター
============================

呼び出し可能オブジェクト
------------------------

### 呼び出し可能

Python では、関数もオブジェクトとして扱われる。また、関数のように丸括弧 `()` を付けて実行できるオブジェクトは**呼び出し可能**（callable）とか、**呼び出し可能型**（callable type）と呼ばれる。呼び出し可能オブジェクトには、次のようなものがある。

  * **関数**:  
  組み込み関数とユーザ定義関数がある。
  * **組み込みメソッド**:  
  リストオブジェクトの `append()` メソッドなど。
  * **クラス**:  
  インスタンス化のために呼び出される。
  * **クラスに定義されたメソッド**:  
  クラス属性に割り当てられた関数。
  * **インスタンス**:  
  クラスが `__call__()` メソッドを定義している場合に限る。この場合にインスタンスを、`x(arg1, ...)` のように呼び出すと、暗黙的に `x.__call__(arg1, ...)` と変換される。

組み込み関数 `callable()` は、唯一の位置引数として渡されたオブジェクトが呼び出し可能であれば `True` を、そうでなければ `False` を返す。

In [ ]:
def my_function():
    print("Hello")

class MyClass:
    def __call__(self):
        print("This is called")

my_obj = MyClass()

assert callable(my_function)
assert callable(my_obj)
my_function()
my_obj()

Hello
This is called


### ユーザ定義関数

ユーザ定義関数オブジェクトは、関数定義の実行時に生成される。関数定義は実行可能な文であり、関数定義を実行すると、現在のローカルな名前空間内で関数名が関数オブジェクトに束縛される（関数本体を実行しない。関数本体は関数が呼び出された時にのみ実行される）。

また、ユーザ定義関数オブジェクトは、ラムダ式の評価によっても生成される。ラムダ式で得られる関数オブジェクトは、関数名の束縛を伴わないことから、**無名関数**（anonymous function）と呼ばれる。ラムダ式の構文は次のとおり。

``` python
lambda [引数リスト]: EXPR
```

引数リストは省略できる。無名関数は `EXPR` の評価結果を返す関数オブジェクトを返す。`EXPR` には、代入式と式のリスト以外の式であれば何でも書ける。

無名関数を変数に代入することもできるが、これは PEP 8 で非推奨とされる。名前に束縛される関数の定義は def 文に統一したほうが読みやすいからである。

ユーザ定義関数オブジェクトは、次のような属性を持つ。

| 特殊属性 | 意味 |
|:--|:----|
| `__name__` | 関数の名前 |
| `__dict__` | 属性辞書 |
| `__doc__` | ドキュメンテーション文字列。なければ `None` |

In [ ]:
f = lambda x, y: x + y  # PEP 8 非推奨
assert f(2, 3) == 5

### フレーム

Python は、関数が呼び出されたときに**フレーム**（frame）と呼ばれる特別なオブジェクトを作成する。

フレームは、関数の単一の呼び出しに関する全ての情報（**実行コンテキスト**）を保持する。その内容には以下の読み取り専用属性からアクセスできる。

| 属性 | 格納される情報 |
|:---|:---|
| `f_locals` | ローカル変数を反映した辞書 |
| `f_globals` | モジュールのグローバルな名前空間（辞書） |
| `f_builtins` | 組み込みの名前が入った名前空間（辞書） |
| `f_back` | 呼び出し元のフレームへのポインタ |
| `f_code` | このフレームで実行されているコードオブジェクト。コードオブジェクトはバイトコードを表現する |
| `f_lasti` | 最後に実行したバイトコードの命令の位置 |

フレームは、Python インタプリタが管理する**実行スタック**（execution stack）上に積まれていく。実行スタックは**コールスタック**（call statck）とも呼ばれる。関数から戻ると、そのフレームは実行スタックから取り除かれる。

![](https://www.plantuml.com/plantuml/png/bP4nJiD044NxVufXAxj8WPqAAIXbxX2bMDbhPuHPhDs9448Ko6UWvm284t2RlGZM16GCSi4Kl_Wp-_veTXl51ZdTTcYuSnIhmBtG7hLhg0tE2Visowz0OnsPDXJjmuwpN8q-i9VioBcEQ-TxDZWv1KzcnnS64mW7Z87L3KLn_INPIAQoNKTg6dg4RPqNQiwjwHsbv_SqlAJXBGqVQNWrUFe1AfP11UsXY_rbO3CMbZDmm2obN4uf_xNk-ZS8NsTVrNVtVdLzFWvKPru1rPbNGdNchR5rlXu_zHC0)

フレームは、処理系が LEGB ルールを実装するのに利用される。実際、関数から戻るとフレームがスタックから取り除かれることによって、関数の名前空間の情報を取得できなくなり、外側のスコープから関数のローカル変数が見えなくなることが実現される。

フレームは内部的に作成される特別なオブジェクトだが、Python 標準ライブラリの `inspect` モジュールが提供する `currentframe()` 関数を使って実行中の関数について作成されたフレームにアクセスすることができる。

フレームの `f_code` 属性で参照されるコードオブジェクトは、バイトコードの状態を取得する読み取り専用属性を多数持っている。このうちの 2 つだけ取り上げる。

| 特殊属性 | 意味 |
|:---|:---|
| `co_name` | 現在のスコープを形成しているコードブロックの名前など |
| `co_freevars` | 現在のスコープが外側のスコープ内で参照している自由変数の名前を含むタプル |

`co_name` 属性が返す値は、次のようになる。

  * 関数の外では `'<module>'`  が返される。IPython では `'<cell line: x>'` のような文字列が返される。
  * 無名関数内では `'<lambda>'` が返される。
  * 関数内では関数名が返される。
  * クラス定義内では、クラス名が返される。
  * クラスのメソッド内では、メソッド名が返される。クラス名は含まれない。

In [ ]:
import inspect

print("関数外:", inspect.currentframe().f_code.co_name)

def f(x=lambda: print("無名関数内:", inspect.currentframe().f_code.co_name)):
    x()
    print("関数内:", inspect.currentframe().f_code.co_name)

f()

class A:
    print("クラス定義内:", inspect.currentframe().f_code.co_name)
    def m(self):
        print("メソッド内:", inspect.currentframe().f_code.co_name)

A().m()

関数外: <cell line: 0>
無名関数内: <lambda>
関数内: f
クラス定義内: A
メソッド内: m


### クロージャー

Python は関数を**第一級オブジェクト**（first-class object）として扱う、非常に関数指向な言語である。プログラミング言語において第一級オブジェクトとは、数値や文字列などの基本的な値と同じように、生成、代入、演算、（引数・戻り値としての）受け渡しといった基本的な操作が制限なしに可能な対象のことをいう。

例えば、Python では、ネストされた関数（内側関数）を外側関数が戻り値として返し、これを別の場所から呼ぶことができる。

内側関数は、その中の自由変数を外側のスコープ（エンクロージングスコープ）から参照できるのであるが、これは外側関数とは別の場所から呼び出されるときでも同様である。

実際、次コードでは `create_adder()` 関数が返す関数を `adder` 変数に保持させ、`create_adder()` 関数の外部で `adder(1)` のように呼び出しているが、自由変数 `n` へのアクセスを保持している。

In [ ]:
def create_adder():
    n = 10

    def function(x):
        return x + n

    return function

adder = create_adder()
assert adder(1) == 11

このように、外側関数で定義された非ローカル変数を外側関数の外部から参照できる内側関数のことを**クロージャー**（closure）または**関数閉包**と呼ぶ。

しかし、クロージャーは、関数の名前空間の寿命を考えると本来は存在しえないものである。というのも、外側関数から戻った時点で、外側関数の名前空間は削除されているからである。LEGB ルールの実装に利用されるフレームも、関数から戻ると実行スタックから取り除かれている。

クロージャーが成立するためには、外側関数のフレームにあった変数、つまりエンクロージングスコープの名前空間の情報を、関数オブジェクトが参照し続ける必要がある。Python はこれを**セルオブジェクト**（cell object）という仕組みで実現している。

Python インタプリタは、外側関数の実行中に、内側関数から参照されるローカル変数（非ローカル変数）を特定し、セルオブジェクトに格納する。外側関数が内側関数を返す際、Python は内側関数のコードと、セルオブジェクトへの参照をラップし、関数オブジェクトとして返す。返された関数オブジェクト（クロージャー）は、呼び出されるたびに、`__closure__` 属性を通じてセルオブジェクトにアクセスし、保存された非ローカル変数の値を取得したり、更新したりする。

次のコードは、クロージャーの自由変数とセルオブジェクトを調査する例である。

In [ ]:
def create_adder():
    n = 10

    def function(x):
        return x + n

    return function

adder = create_adder()
print(adder.__code__.co_freevars)
print(adder.__closure__)

('n',)
(<cell at 0x782209c53730: int object at 0xb1d608>,)


ラムダ式で定義された関数も、通常の def 文で定義された関数と同様に、クロージャーを形成できる。

ラムダ式の本体にはラムダ式を置けるので、ラムダ式のネストが可能である。

In [ ]:
div = lambda x: lambda y: x / y
assert div(10)(2) == 5.0

外側の `lambda x:...` は、内側の `lambda y: x / y` によって生成された関数オブジェクトを返す。`lambda y: x / y` にとって `x` は自由変数である。

`div(10)` を呼び出すと、自由変数 `x` に 10 を束縛した `lambda y: 10 / y` に相当する関数オブジェクトが生成されるため、`div(10)(2)` の呼び出しでは `10 / 2` の計算が行われる。このとき、`x` は外側のラムダ式の評価が完了した後も参照可能であり、これは関数オブジェクトがクロージャーとして自由変数を内部に保持する仕組みによるものである。

関数定義のネストでは nonlocal 宣言によって自由変数を共有し変更することができたが、形成されるクロージャーの間でも自由変数を共有する際には、外側関数の名前空間は関数が実行されるたびに新しく生成されることに留意しなければならない。例えば次のコードでは、自由変数をクロージャーがインクリメントするが、外側関数を実行するたびにその変数は初期化される:

In [ ]:
def create_counter():
    count = 0

    def counter():
        nonlocal count
        count += 1
        return count

    return counter

counter1 = create_counter()
print("counter1: {}".format(counter1()))
print("counter1: {}".format(counter1()))
print("counter1: {}".format(counter1()))
counter2 = create_counter()
print("counter2: {}".format(counter2()))
print("counter2: {}".format(counter2()))
print("counter2: {}".format(counter2()))

counter1: 1
counter1: 2
counter1: 3
counter2: 1
counter2: 2
counter2: 3


複数のクロージャーが同じ自由変数を共有するには、単に外側関数が複数のクロージャーを生成すればよい。

In [ ]:
def create_counter():
    count = 0

    def counter1():
        nonlocal count
        count += 1
        return count

    def counter2():
        nonlocal count
        count += 1
        return count

    return counter1, counter2

counter1, counter2 = create_counter()
print("counter1: {}".format(counter1()))
print("counter2: {}".format(counter2()))
print("counter1: {}".format(counter1()))
print("counter2: {}".format(counter2()))

counter1: 1
counter2: 2
counter1: 3
counter2: 4


ループで複数のクロージャーを生成する場合、反復変数はループが終了した後の値でクロージャーに共有されることに注意する:

In [ ]:
def create_adders():
    adders = []
    for n in range(1, 4):
        adders.append(lambda x: x + n)
    return adders

adder1, adder2, adder3 = create_adders()
print("adder1: {}".format(adder1(10)))
print("adder2: {}".format(adder2(10)))
print("adder3: {}".format(adder3(10)))

adder1: 13
adder2: 13
adder3: 13


このコードでは、`n` はクロージャーが呼び出された時に評価されるため、このような動作になる。

クロージャーの定義が実行された時点での反復変数の値を使いたい場合は、反復変数をクロージャーのデフォルト値付き引数とすればよい。これは、デフォルト値が関数定義を実行する時に評価されることを利用している。

In [ ]:
def create_adders():
    adders = []
    for n in range(1, 4):
        adders.append(lambda x, n=n: x + n)
    return adders

adder1, adder2, adder3 = create_adders()
print("adder1: {}".format(adder1(10)))
print("adder2: {}".format(adder2(10)))
print("adder3: {}".format(adder3(10)))

adder1: 11
adder2: 12
adder3: 13


デコレーター
------------

### 関数デコレーター

Python において、関数は第一級オブジェクトであるから、他の関数の引数として渡したり、ネストされた関数を戻り値として返すことができる。

他の関数の引数として渡される関数を**コールバック関数**（callback function）と呼び、反対に、関数を引数に取る関数を**高階関数**（higher-order function）と呼ぶ。

一方、関数を返す関数を**ファクトリ関数**と呼ぶ。とくに、関数を返す高階関数を、Python では**関数デコレーター**または簡単に**デコレーター**（decorator）と呼ぶ。ある関数にデコレーターを適用することを「デコレートする」と表現する。

デコレーターを使うことにより、既存の関数 `my_func()` そのものの中身を変えず、関数の前後に処理を追加して実行するということができる：

In [ ]:
def decorator(func):
    def wrap_function():
        """wrapper function"""
        print("some kind of pre-processing")
        ret = func()
        print("some post-processing")
        return ret

    return wrap_function

def my_func():
    """original function"""
    print("original function")

my_func = decorator(my_func)
my_func()

some kind of pre-processing
original function
some post-processing


この `decorator()` 関数はデコレーターであり、内部関数として `wrap_function()` を定義し返している。`decorator()` の引数 `func` は関数であり、これを `wrap_function()` の中で呼び出すことで、`wrap_function()` は `func` 関数の実行前後に処理を追加するものとなっている。ここで `func` は `wrap_function()` の自由変数であるが、クロージャーの仕組みにより `decorator()` 関数の外部で `wrap_function()` を呼び出しても `func` を参照することができる。

`wrap_function()` のような関数は **wrapper 関数**と呼ばれ、`func`（引数として渡される関数）は **wrapped 関数**と呼ばれる。

最後の 2 行で変数 `my_func` に `decorator()` 関数の戻り値（内部関数である `wrap_function()`）を代入し、 `my_func()` を呼び出している。この呼び出しは、実際には `wrap_function()` を実行することになる。

ただし、もとの関数 `my_func()` の関数定義と、変数 `my_func` への関数オブジェクト（クロージャー）の代入が別になっているのは、問題を引き起こす可能性がある。というのは、その間に `my_func()` の呼び出しを書いてしまうと、もとの関数 `my_func()` が実行できてしまうからである。そこで、Python では上記のコードと等価な別の書き方をサポートしている：

In [ ]:
def decorator(func):
    def wrap_function():
        """wrapper function"""
        print("some kind of pre-processing")
        ret = func()
        print("some post-processing")
        return ret

    return wrap_function

@decorator
def my_func():
    """original function"""
    print("original function")

my_func()

some kind of pre-processing
original function
some post-processing


&commat; 構文は**デコレータ式**（decorator expression）と呼ばれるシンタックスシュガーで、上記のように関数定義の直前に `@decorator` と書くと、Python インタプリタは、暗黙のうちに関数定義の直後に

``` python
my_func = decorator(my_func)
```

を自動で挿入してくれる。これにより、もとの関数 `my_func()` そのものを実行できる余地がなくなる。

複数のデコレーターを適用することもできる:

``` python
@decorator1
@decorator2
@decorator3
def func():
    pass
```

デコレータ式は「下から上へ」再帰的に展開されるため、このコードは次の代入文と等価である：

``` python
func = decorator1(decorator2(decorator3(func)))
```

In [ ]:
def decorator1(func):
    print("decorator1 applied")
    return func

def decorator2(func):
    print("decorator2 applied")
    return func

def decorator3(func):
    print("decorator3 applied")
    return func

@decorator1
@decorator2
@decorator3
def func():
    pass

decorator3 applied
decorator2 applied
decorator1 applied


### wrapper 関数の汎用化

デコレーターを定義するとき、内部の wrapper 関数を、ある 1 つの wrapped 関数に依存した書き方（つまり、その関数の引数や戻り値を決め打ちした書き方）をする場合、せっかく定義したデコレーターが他の関数に適用できなくなる。あるいは、将来 wrapped 関数を変更すると、デコレーターの変更も必要となる。そこで、wrapper 関数は、できるだけ wrapped 関数の引数や戻り値を決め打ちして実装することがないようにすべきである。

一方で、wrapper 関数の引数は、wrapped 関数の引数を受け付けるものでなければならない。wrapper 関数の外では wrapped 関数の呼び出しを wrapper 関数の呼び出しに置き換え、しかも、wrapper 関数の中では wrapped 関数を呼び出すからである。前出の wrapper 関数 `wrap_function()` は、wrapped 関数が引数を取らないと決め打ちしていたので、引数を取る wrapped 関数に対しては、TypeError 例外が発生してしまう。

In [ ]:
@decorator
def my_func_with_arg(arg):
    """original function with argument"""
    print(f"original function with argument {arg}")

try:
    my_func_with_arg(1)
except TypeError as err:
    print(f"{type(err).__name__}: {err}")

TypeError: decorator.<locals>.wrap_function() takes 0 positional arguments but 1 was given


wrapped 関数の引数がどのようなものであっても、それを受け付けることができる wrapper 関数を定義するには、可変長引数を使うとよい。wrapper 関数の引数を `*args, **kwargs` と定義し、中で呼び出す wrapped 関数には アンパック `*args, **kwargs` を渡すようにすれば、すべての wrapped 関数の引数に対応できる：

In [ ]:
def decorator(func):
    def wrap_function(*args, **kwargs):
        """wrapper function"""
        print("some kind of pre-processing")
        ret = func(*args, **kwargs)
        print("some post-processing")
        return ret

    return wrap_function

@decorator
def my_func():
    """original function"""
    print("original function")

@decorator
def my_func_with_arg(arg):
    """original function with argument"""
    print(f"original function with argument {arg}")

my_func()
my_func_with_arg(1)

some kind of pre-processing
original function
some post-processing
some kind of pre-processing
original function with argument 1
some post-processing


### 引数付きデコレーター

デコレータ式は、式のリスト以外の任意の式を記述できる:

``` python
@<式>
```

したがって、&commat; 構文に「関数の呼び出し」も記述できる。このとき、引数を渡すこともできる。これは「引数付きデコレーター」と呼ばれる。「引数付きデコレーター」は、評価されると直下の関数をデコレートするデコレーターを返すことに注意する。つまり、「引数付きデコレーター」となる関数は、次のように内部関数として「もとの関数を直接デコレートするデコレーター」を定義し、さらにその内部に wrapper 関数を定義するものとなる。

In [ ]:
def decorator(option):
    def inner_decorator(func):
        def wrap_function(*args, **kwargs):
            """wrapper function"""
            print(f"some kind of pre-processing with {option=}")  # option の値を利用できる
            ret = func(*args, **kwargs)
            print(f"some post-processing with {option=}")  # option の値を利用できる
            return ret

        return wrap_function

    return inner_decorator

@decorator("foo")
def my_func(arg):
    """original function"""
    print(f"original function: {arg=}")

my_func(1)

some kind of pre-processing with option='foo'
original function: arg=1
some post-processing with option='foo'


`decorator()` の仮引数 `option` は、 `wrap_function()` 関数にとっての自由変数であり、 `inner_decorator()` 関数にとっての自由変数でもある。このようにクロージャーは他のクロージャー内にネストすることができ、複数レベルのスコープと自由変数を持つことができるので、`option` は `wrap_function()` 関数の呼び出しにおいて参照可能である。

`my_func(arg)` の呼び出しは、`decorator(option)(my_func)(arg)` の呼び出しと等価である。

呼び出し可能型を使って次のように書くこともできる。

In [ ]:
class Decorator:
    def __call__(self, option):
        self.option = option
        return self.inner_decorator

    def inner_decorator(self, func):
        def wrap_function(*args, **kwargs):
            """wrapper function"""
            print(f"some kind of pre-processing with {self.option=}")  # self.option の値を利用できる
            ret = func(*args, **kwargs)
            print(f"some post-processing with {self.option=}")  # elf.option の値を利用できる
            return ret
        return wrap_function

decorator = Decorator()

@decorator("foo")
def my_func(arg):
    """original function"""
    print(f"original function: {arg=}")

my_func(1)

some kind of pre-processing with self.option='foo'
original function: arg=1
some post-processing with self.option='foo'


### functools.wraps

デコレーターを適用すると、オリジナル関数の `__name__` 属性と `__doc__` 属性は wrapper 関数のものに置き換えられてしまう。

In [ ]:
def decorator(func):
    def wrap_function(*args, **kwargs):
        """wrapper function"""
        return func(*args, **kwargs)
    return wrap_function

@decorator
def my_func():
    """original function"""
    print("original function")

assert my_func.__name__ == "wrap_function"
assert my_func.__doc__ == "wrapper function"

`help()` 関数やエディタの入力支援はこれらの属性を表示するので、混乱の原因となる。置き換えられないようにするには、標準ライブラリの `functools` モジュールが提供する `functools.wraps()` 関数を wrapper 関数に適用すればよい。`functools.wraps()` 関数は wrapped 関数を引数に取るので、引数付きデコレーターとして、次のように wrapper 関数に適用する。

In [ ]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrap_function(*args, **kwargs):
        """wrapper function"""
        return func(*args, **kwargs)
    return wrap_function

@decorator
def my_func():
    """original function"""
    print("original function")

assert my_func.__name__ == "my_func"
assert my_func.__doc__ == "original function"

なお、`functools.update_wrapper()` 関数も提供されており、第 1 引数の関数の属性を第 2 引数の関数からコピーするように更新する。

`functools.wraps(wrapped)` は、次のコードで定義される関数を使った `my_wraps(wrapped)` と等価である。

``` python
def my_wraps(wrapped):
    """functools.wrapsと等価な関数"""
    return lambda wrapper: functools.update_wrapper(wrapper, wrapped)
```

メモ化
------

`functools` モジュールは、便利なデコレーターを提供している。

``` python
functools.cache(user_function)
```

これは、関数 `user_function` が[メモ化](https://ja.wikipedia.org/wiki/メモ化)された関数を返すデコレーターである。Python 3.9 で追加された。Python 3.8 以前なら、`functools.lru_cache(maxsize=None)` が同等の関数を返す。

メモ化された関数は、引数と戻り値をキャッシュに記憶しておき、2 回目以降の呼び出しに同じ引数を見つけたときは記憶してある値を返す。これにより、同じ計算を繰り返すことがなくなって効率が良くなる。

ただし、キャッシュの性質上、以下の点に注意する。

  * キャッシュには、引数をキー、戻り値をキーの値とする辞書が使われるので、**引数はハッシュ可能な値でなければならない**。キーワード引数の順序が異なる場合は、異なるキャッシュとして保存されるため、メモ化の効果がない。
  * メモ化される関数は**参照透過性**（同じ引数に対して同じ戻り値を返すこと）を持つ必要がある。参照透過性を持たない関数のメモ化では、その関数の本来の結果を受け取ることができなくなってしまう。
  * キャッシュはプログラムが終了すると失われる。

In [ ]:
from functools import cache

@cache
def func(num):
    print(f"{num=}: 関数の中に入って処理しています")
    return num + 1

r1 = func(100)
r2 = func(200)
r3 = func(100)  # キャッシュから値を返すので、実際には関数本体が実行されない
assert [r1, r2, r3] == [101, 201, 101]  # r3 はキャッシュからの値

num=100: 関数の中に入って処理しています
num=200: 関数の中に入って処理しています


メモ化は、関数に同じ引数が何度も渡されるようなアルゴリズムで使用すると大きな効果を発揮する。そのようなアルゴリズムの例が[動的計画法](https://ja.wikipedia.org/wiki/動的計画法)による再帰的アルゴリズムである。以下のコードでは、`fibonacci()` は[フィボナッチ数](https://ja.wikipedia.org/wiki/フィボナッチ数)を返す再帰関数であり（初項は 1 とする）、メモ化を使用しない場合の実行時間を計測している。

In [ ]:
def fibonacci(x):
  if x==1 or x==2:
    return 1
  else:
    return fibonacci(x-1) + fibonacci(x-2)

%timeit -n 30 -r 3 fibonacci(20)

10.4 ms ± 1.34 ms per loop (mean ± std. dev. of 3 runs, 30 loops each)


この `fibonacci()` 関数をメモ化して実行時間を計測すると、1/10000 近くに実行時間が短縮化されることがわかる:

In [ ]:
from functools import cache

@cache
def fibonacci(x):
  if x==1 or x==2:
    return 1
  else:
    return fibonacci(x-1) + fibonacci(x-2)

%timeit -n 30 -r 3 fibonacci(20)

The slowest run took 11.41 times longer than the fastest. This could mean that an intermediate result is being cached.
521 ns ± 557 ns per loop (mean ± std. dev. of 3 runs, 30 loops each)


クラスデコレーター
------------------

Python では、クラスは呼び出し可能なオブジェクトである。クラスオブジェクトを引数にとり、クラスオブジェクトを返す関数を**クラスデコレーター**という。

In [ ]:
def class_decorator(cls):
    cls.x = "sample"
    return cls

class MyClass:
    """original class"""
    pass

MyClass = class_decorator(MyClass)

# インスタンスを生成
m = MyClass()
print(f"{m.x=}")

m.x='sample'


`@` を使うデコレータ式は、関数と同様にクラスもサポートしている。

In [ ]:
def class_decorator(cls):
    cls.x = "sample"
    return cls

@class_decorator
class MyClass:
    """original class"""
    pass

# インスタンスを生成
m = MyClass()
print(f"{m.x=}")

m.x='sample'


クラスデコレーターでできることは、メタクラスで `__init__()` をオーバーライドする方法でも実現できる。

複数のデコレーターを適用することもできる:

``` python
@f1(arg)
@f2
class Foo: pass
```

は、だいたい次と等価である。

``` python
class Foo: pass
Foo = f1(arg)(f2(Foo))
```